### Homework

In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup
from splinter import Browser
import os
import pandas as pd
import time
import tweepy

In [2]:
#pointing to the directory where chromedriver exists
executable_path = {"executable_path":"C:\chromedriver_win32\chromedriver"}
browser = Browser("chrome", **executable_path, headless = False)

#### NASA Mars News

In [3]:
#visiting the page
url = "https://mars.nasa.gov/news/"
browser.visit(url)

#using bs to write it into html
html = browser.html
soup = bs(html,"html.parser")

#extract info
news_title = soup.find("div",class_="content_title").text
news_p = soup.find("div", class_="rollover_description_inner").text
print(f"Title: {news_title}")
print(f"Paragraph: {news_p}")

Title: Nine Finalists Chosen in NASA's Mars 2020 Rover Naming Contest
Paragraph: Nine finalists have been chosen in the essay contest for K-12 students across U.S. to name NASA's next Mars rover. Now you can help by voting for your favorite. 


#### JPL Mars Space Images - Featured Image

In [4]:
# Visit the JPL Mars URL
url2 = "https://jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url2)

# Scrape the browser into soup and use soup to find the image of mars
# Save the image url to a variable called `img_url`
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
image = soup.find("img", class_="thumb")["src"]
img_url = "https://jpl.nasa.gov"+image
featured_image_url = img_url
# Use the requests library to download and save the image from the `img_url` above
import requests
import shutil
response = requests.get(img_url, stream=True)
with open('img.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
    
# Display the image with IPython.display
from IPython.display import Image
Image(url='img.jpg')

#### Mars Weather


In [7]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

html = browser.html
weather_soup = BeautifulSoup(html, 'html.parser')
mars_weather_tweet = weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
mars_weather

AttributeError: 'NoneType' object has no attribute 'find'

In [9]:
weather_soup

<html dir="ltr" lang="en" style="font-size: 15px;"><head><meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
<link href="//abs.twimg.com" rel="preconnect"/>
<link href="//api.twitter.com" rel="preconnect"/>
<link href="//pbs.twimg.com" rel="preconnect"/>
<link href="//t.co" rel="preconnect"/>
<link href="//video.twimg.com" rel="preconnect"/>
<link href="//abs.twimg.com" rel="dns-prefetch"/>
<link href="//api.twitter.com" rel="dns-prefetch"/>
<link href="//pbs.twimg.com" rel="dns-prefetch"/>
<link href="//t.co" rel="dns-prefetch"/>
<link href="//video.twimg.com" rel="dns-prefetch"/>
<link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/web/polyfills.0894f704.js" nonce="" rel="preload"/>
<link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/web/vendors~main.a5f70344.js" nonce="" rel="preload"/>
<link as="script" crossorigin="anonymous" 

In [56]:
# Twitter API Keys
from key_vault import (consumer_key, consumer_secret, access_token, access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
target_user = "marswxreport"
full_tweet = api.user_timeline(target_user , count = 1)
mars_weather=full_tweet[0]['text']
mars_weather

TweepError: [{'code': 89, 'message': 'Invalid or expired token.'}]

#### Mars Facts

In [21]:
url3 = "http://space-facts.com/mars/"
browser.visit(url3)

# place data into a dataframe, clean it up and output it into an HTML table
import pandas as pd 
grab=pd.read_html(url3)
mars_data=pd.DataFrame(grab[0])
mars_data.columns=['Mars','Data']
mars_table=mars_data.set_index("Mars")
marsdata = mars_table.to_html(classes='marsdata')
marsdata=marsdata.replace('\n', ' ')
marsdata

'<table border="1" class="dataframe marsdata">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Data</th>     </tr>     <tr>       <th>Mars</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.39 × 10^23 kg (0.11 Earths)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.38 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-87 to -5 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>'

#### Mars Hemispheres


In [32]:
# Visit the USGS Astogeology site and scrape pictures of the hemispheres
url4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url4)

# Use splinter to loop through the 4 images and load them into a dictionary
import time 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
mars_hemis=[]

# loop through the four tags and load the data to the dictionary
for i in range (4):
    time.sleep(5)
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    partial = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ partial
    dictionary={"title":img_title,"img_url":img_url}
    mars_hemis.append(dictionary)
    browser.back()
    
print(mars_hemis)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


Done!